In [ ]:
!pip install -q transformers accelerate bitsandbytes peft
from google.colab import drive
drive.mount('/content/drive')

import json

with open("/content/drive/MyDrive/Apziva/train_dataset.json", "r", encoding="utf-8") as f:
    train_dataset = json.load(f)

print(f"Loaded {len(train_dataset)} training samples")
print(train_dataset[:3])  # 查看前 3 个样本



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.9 MB/s eta 0:00:00
Mounted at /content/drive
Loaded 1225 training samples
[{'instruction': 'Compare two job titles for HR role relevance and decide which candidate is more fit.', 'input': 'A: Admissions Representative at Community medical center long beach\nB: Student', 'output': 'Student'}, {'instruction': 'Compare two job titles for HR role relevance and decide which candidate is more fit.', 'input': 'A: Admissions Representative at Community medical center long beach\nB: Native English Teacher at EPIK (English Program in Korea)', 'output': 'Admissions Representative at Community medical center long beach'}, {'instruction': 'Compare two job titles for HR role relevance and decide which candidate is more fit.', 'input': 'A: Admissions Representative at Community medical center long beach\nB: Always set them up for Success', 'output': 'Admissions Representative at Community medical center long beach'}]


In [ ]:
from huggingface_hub import login
login()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # FP16 节省显存
)

# LoRA 配置
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],  # Qwen 常用投影矩阵
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


In [ ]:
from torch.utils.data import Dataset

class PairwiseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = f"{item['instruction']}\n{item['input']}\nAnswer:"
        labels = item['output']

        input_enc = self.tokenizer(text, truncation=True, max_length=self.max_length, padding="max_length", return_tensors="pt")
        label_enc = self.tokenizer(labels, truncation=True, max_length=32, padding="max_length", return_tensors="pt")

        input_ids = input_enc["input_ids"].squeeze()
        attention_mask = input_enc["attention_mask"].squeeze()

        # labels: input set as -100，only train output token
        label_ids = -100 * torch.ones_like(input_ids)
        label_len = min(label_enc["input_ids"].size(1), len(label_ids))
        label_ids[-label_len:] = label_enc["input_ids"].squeeze()[:label_len]

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": label_ids
        }

train_data = PairwiseDataset(train_dataset, tokenizer)
print(train_data)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Apziva/qwen_lora_pairwise",
    per_device_train_batch_size=1,      # small batch to avoid OOM
    gradient_accumulation_steps=4,     # set gradient accumulation
    learning_rate=2e-4,
    fp16=True,
    num_train_epochs=5,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    remove_unused_columns=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:600: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
5,13.092653
10,12.189125
15,8.042621
20,4.552857
25,3.206866
30,2.425964
35,2.370059
40,2.082355
45,2.114337
50,1.967975


TrainOutput(global_step=1535, training_loss=0.5168978185468757, metrics={'train_runtime': 1474.8761, 'train_samples_per_second': 4.153, 'train_steps_per_second': 1.041, 'total_flos': 2.612241235968e+16, 'train_loss': 0.5168978185468757, 'epoch': 5.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Apziva/qwen_lora_pairwise")
tokenizer.save_pretrained("/content/drive/MyDrive/Apziva/qwen_lora_pairwise")
print("LoRA model saved successfully!")

LoRA model saved successfully!


In [ ]:
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Apziva/qwen_lora_pairwise")
base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Apziva/qwen_lora_pairwise")
model.eval()

# 示例比较
title_a = "Aspiring Human Resources Professional"
title_b = "Sales Associate"

prompt = f"Compare two job titles for HR role relevance:\nA: {title_a}\nB: {title_b}\nAnswer:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=20)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Model output:", result)

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Model output: Compare two job titles for HR role relevance:
A: Aspiring Human Resources Professional
B: Sales Associate
Answer: A


In [ ]:
import pandas as pd
df_path = "/content/drive/MyDrive/Apziva/ProjectC.csv"
df = pd.read_csv(df_path).drop_duplicates(subset=["job_title"])
print("Loaded df shape:", df.shape)
print(df.head())

def lora_pairwise_winner(title_a, title_b):

    prompt = (
        "Compare two job titles for HR role relevance.\n"
        f"A: {title_a}\n"
        f"B: {title_b}\n"
        "Answer with only 'A' or 'B'.\n"
        "Answer:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=2,
        do_sample=False,   # key
        temperature=0.0,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = decoded.split("Answer:")[-1].strip()
    if answer.startswith("A"):
        return title_a
    elif answer.startswith("B"):
        return title_b
    else:
        return "Tie"


def pairwise_sort(df, title_col="job_title"):
    titles = df[title_col].tolist()
    n = len(titles)
    scores = [0] * n
    pairwise_results = []

    for i in range(n):
        print(f"Processing {i+1}/{n} titles...")
        for j in range(i+1, n):
            winner = lora_pairwise_winner(titles[i], titles[j])

            pairwise_results.append({
                "A": titles[i],
                "B": titles[j],
                "winner": winner
            })

            if winner == titles[i]:
                scores[i] += 1
            elif winner == titles[j]:
                scores[j] += 1
            else:  # Tie
                scores[i] += 0.5
                scores[j] += 0.5

    sorted_df = pd.DataFrame({
        "job_title": titles,
        "score": scores
    }).sort_values(by="score", ascending=False).reset_index(drop=True)

    pairwise_df = pd.DataFrame(pairwise_results)

    return sorted_df, pairwise_df

sorted_df, pairwise_df = pairwise_sort(df, title_col="job_title")

print("Top 10 sorted job titles:")
print(sorted_df)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loaded df shape: (52, 5)
   id                                          job_title  \
0   1  2019 C.T. Bauer College of Business Graduate (...   
1   2  Native English Teacher at EPIK (English Progra...   
2   3              Aspiring Human Resources Professional   
3   4             People Development Coordinator at Ryan   
4   5    Advisory Board Member at Celal Bayar University   

                              location connection  fit  
0                       Houston, Texas         85  NaN  
1                               Kanada      500+   NaN  
2  Raleigh-Durham, North Carolina Area         44  NaN  
3                        Denton, Texas      500+   NaN  
4                       İzmir, Türkiye      500+   NaN  
Processing 1/52 titles...
Processing 2/52 titles...
Processing 3/52 titles...
Processing 4/52 titles...
Processing 5/52 titles...
Processing 6/52 titles...
Processing 7/52 titles...
Processing 8/52 titles...
Processing 9/52 titles...
Processing 10/52 titles...
Processing 